In [1]:
import time
import os

import torch

import pytorch_mask_rcnn as pmr


use_cuda = True
train_num_samples = 30
val_num_samples = 20
ckpt_path = '../checkpoint.pth'
data_root = 'E:/PyTorch/data/VOC2012'
classes = (
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
    'dog', 'horse', 'motorbike', 'person', 'pottedplant',
    'sheep', 'sofa', 'train', 'tvmonitor') # 20 classes

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('cuda: {}\nuse_cuda: {}\n{} GPU(s) available'.format(torch.cuda.is_available(), use_cuda, torch.cuda.device_count()))
print('\ndevice: {}'.format(device))

trainset = pmr.VOCDataset(data_root, 'train', True, device=device) # len=1463
valset = pmr.VOCDataset(data_root, 'val', False, device=device) # len=1444

indices = torch.randperm(len(trainset)).tolist()
trainset = torch.utils.data.Subset(trainset, indices[:train_num_samples])
indices = torch.randperm(len(valset)).tolist()
valset = torch.utils.data.Subset(valset, indices[:val_num_samples])

torch.manual_seed(3)
model = pmr.maskrcnn_resnet50(False, len(classes) + 1).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

cuda: True
use_cuda: True
1 GPU(s) available

device: cuda


In [2]:
if os.path.exists(ckpt_path):
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    del checkpoint['model_state_dict']
    del checkpoint['optimizer_state_dict']
    torch.cuda.empty_cache()
else:
    checkpoint = dict(num_batches=0)
print('already trained: {}'.format(checkpoint['num_batches']))

since = time.time()
# ------------------train---------------------
model.train()
for i, data in enumerate(trainset):
    optimizer.zero_grad()
    losses = model(*data)
    #print(' '.join(str(round(l.item(), 3)) for l in losses.values()))
    loss = sum(losses.values())
    loss.backward()
    optimizer.step()
# ------------------train---------------------
print('total time: {:.2f} s'.format(time.time() - since))

checkpoint['model_state_dict'] = model.state_dict()
checkpoint['optimizer_state_dict'] = optimizer.state_dict()
checkpoint['num_batches'] += i + 1
torch.save(checkpoint, ckpt_path)

num_batches = checkpoint['num_batches']
del checkpoint
torch.cuda.empty_cache()

print('\nalready trained: {}'.format(num_batches))

already trained: 60
total time: 15.79 s

already trained: 90


In [3]:
since = time.time()
# ------------------val---------------------
model.eval()
for image in valset:
    with torch.no_grad():
        results = model(image)
        
    if False:
        print(results['boxes'])
        print('  '.join(classes[l] for l in results['labels'].tolist()))
        print('  '.join('{:.2f}'.format(p) for p in results['scores'].tolist()))
        pmr.show(image, results['boxes'])
# ------------------val---------------------
print('total time: {:.2f} s'.format(time.time() - since))

total time: 5.13 s
